<a href="https://colab.research.google.com/github/aarsanjani/meansquares/blob/master/version2/preprocessing/AllStates_MergeSupetSetData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Goal

This colab merges for All States
* combined dataset and Maryland dataset
* and creates a new dataset called '**Superset Dataset**"

# Mount drive

In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [2]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"]="2"
import warnings; warnings.simplefilter('ignore')

In [ ]:
#drive.flush_and_unmount(600000)

In [3]:
!ls '/content/drive/Shared drives/CMPE 295- Master Project/projectdata-2021'

 All_CovidDataFeb20.csv
 alldatasets_Feb20.csv
 All_GoogleMobilityDataFeb20.csv
 All_SocialDistancingDataFeb20.csv
 AllStates_Superset_dataset_Feb20.csv
 CA_alldatasets_Feb05.csv
 CA_alldatasets_Jan13.csv
 CA_CombinedData_Labeled_CDCGuideline_Feb05.csv
 CA_CombinedData_Labeled_CDCGuideline_Feb06.csv
 CA_CombinedData_Labeled_CDCGuideline_Jan15.csv
 CA_combined_Feb04.csv
 CA_CovidDataJan10.csv
 CA_dataset_Feb11.csv
 CA_GoogleMobilityDataJan10.csv
 CA_MarylandData_Labeled_CDCGuideline_Feb05.csv
 CA_MarylandData_Labeled_CDCGuideline_Feb06.csv
 CA_MarylandData_Labeled_CDCGuideline_Jan15.csv
 CA_processedMarylandData_Feb04.csv
 CA_processedMarylandData_Feb05.csv
 CA_processedMarylandData_Feb12.csv
 CA_processedMarylandData_Jan11.csv
 CA_SocialDistancingDataJan10.csv
 CA_socialDistancingInertiaData_Feb04.csv
 CA_SuperData_Labeled_CDCGuideline_Feb13.csv
 CA_Superset_bins_dataset_Feb20.csv
 CA_Superset_dataset_Feb12.csv
 computed_CA_mobility_data.csv
 data-mask_for_all.csv
 election_results_202

# Import package

In [1]:
import pandas as pd
import seaborn as sns
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler as ss
import itertools
from scipy import linalg
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import export_graphviz
import numpy as np
import matplotlib.pyplot as plt

# Load Data Combined Data

In [2]:
combined_data = pd.read_csv('/content/drive/Shared drives/CMPE 295- Master Project/projectdata-2021/alldatasets_Feb20.csv',low_memory=False)



In [3]:
print(combined_data.shape)


(3580280, 15)


In [4]:
combined_data.head(2)

,Date,Province_State,FIPS,New cases,County Name,population,mask_rule_severity,m50,m50_index,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
0,2020-03-03,Montana,30001,0,Beaverhead County,9453,0.0,1.139,86,9.0,7.0,0.0,0.0,-2.0,0.0
1,2020-03-04,Montana,30001,0,Beaverhead County,9453,0.0,1.480,113,14.0,7.0,0.0,0.0,3.0,0.0


#  Load Preprocessed Social Distancing Inertia Data for All States[Maryland University]

In [5]:
maryland_sd_inertia = pd.read_csv('/content/drive/Shared drives/CMPE 295- Master Project/projectdata-2021/maryland_inertia_data_allstates_Feb12.csv',low_memory=False)
maryland_sd_inertia.head()

,State_Name,PopDensity,countyFIPS,County Name,State,population,CTFIPS,CTNAME,STFIPS,Social.distancing.index,X..staying.home,Trips.person,X..out.of.county.trips,X..out.of.state.trips,Miles.person,Work.trips.person,Non.work.trips.person,New.COVID.cases,Population,X..change.in.consumption,date,Transit.mode.share,X..people.older.than.60,Median.income,X..African.Americans,X..Hispanic.Americans,X..Male,Population.density,Employment.density,X..hot.spots.1000.people,Hospital.beds.1000.people,ICUs.1000.people,X..contact.tracing.workers.1000.people,New.cases.1000.people,Active.cases.1000.people,X.days..decreasing.COVID.cases,X..hospital.bed.utilization,Testing.capacity,Tests.done.1000.people,X..ICU.utilization,Ventilator.shortage,Imported.COVID.cases,COVID.exposure.1000.people,X.days..decreasing.ILI.cases,Unemployment.claims.1000.people,Unemployment.rate,X..working.from.home,Cumulative.inflation.rate,COVID.death.rate
0,Alabama,35.853419,1001,Autauga County,AL,55869,1001,Autauga County,1,54,27,2.25,38.8,1.2,24.7,0.25,2.00,0,55601,-22.5,2020-01-01,0.11,19,58786,19.0,2.8,48.68,91,19,101,3.79,0.33,0.025,0.0,0.0,0,47.1,0.0,0.0,0.0,0,0,0.0,0,0.9,2.7,3.2,0.3,0.0
1,Alabama,35.853419,1001,Autauga County,AL,55869,1001,Autauga County,1,25,18,2.83,43.5,0.9,37.6,0.62,2.21,0,55601,-14.3,2020-01-02,0.11,19,58786,19.0,2.8,48.68,91,19,101,3.79,0.33,0.025,0.0,0.0,0,47.1,0.0,0.0,0.0,0,0,0.0,0,0.9,2.7,3.2,0.3,0.0
2,Alabama,35.853419,1001,Autauga County,AL,55869,1001,Autauga County,1,15,15,3.18,42.2,1.0,40.3,0.62,2.57,0,55601,-0.4,2020-01-03,0.11,19,58786,19.0,2.8,48.68,91,19,101,3.79,0.33,0.025,0.0,0.0,0,47.1,0.0,0.0,0.0,0,0,0.0,0,0.9,2.7,3.2,0.3,0.0
3,Alabama,35.853419,1001,Autauga County,AL,55869,1001,Autauga County,1,33,20,2.85,43.9,1.2,29.5,0.33,2.52,0,55601,-2.3,2020-01-04,0.11,19,58786,19.0,2.8,48.68,91,19,101,3.79,0.33,0.025,0.0,0.0,0,47.1,0.0,0.0,0.0,0,0,0.0,0,0.9,2.7,3.2,0.3,0.0
4,Alabama,35.853419,1001,Autauga County,AL,55869,1001,Autauga County,1,40,22,2.56,42.8,1.2,30.9,0.23,2.33,0,55601,-9.7,2020-01-05,0.11,19,58786,19.0,2.8,48.68,91,19,101,3.79,0.33,0.025,0.0,0.0,0,47.1,0.0,0.0,0.0,0,0,0.0,0,0.9,2.7,3.2,0.3,0.0


In [6]:
maryland_sd_inertia['State'].unique()

array(['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'DC', 'FL', 'GA',
       'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA',
       'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY',
       'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',
       'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY'], dtype=object)

# Superset Data: Combining Maryland and combined datasets

In [7]:
print(combined_data.shape)
print(maryland_sd_inertia.shape)

print("min-max combined data:: ", combined_data.Date.min(),combined_data.Date.max())

print("min-max maryland data:: ", maryland_sd_inertia.date.min(),maryland_sd_inertia.date.max())

(3580280, 15)
(1140546, 49)
min-max combined data::  2020-03-01 2021-02-16
min-max maryland data::  2020-01-01 2020-12-28


In [8]:
All_state_superset_data = pd.merge(combined_data, maryland_sd_inertia, how='inner',left_on=['Date','FIPS'],right_on=['date','countyFIPS'])
print(All_state_superset_data.shape)

All_state_superset_data.head(5)


(3101381, 64)


,Date,Province_State,FIPS,New cases,County Name_x,population_x,mask_rule_severity,m50,m50_index,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,State_Name,PopDensity,countyFIPS,County Name_y,State,population_y,CTFIPS,CTNAME,STFIPS,Social.distancing.index,X..staying.home,Trips.person,X..out.of.county.trips,X..out.of.state.trips,Miles.person,Work.trips.person,Non.work.trips.person,New.COVID.cases,Population,X..change.in.consumption,date,Transit.mode.share,X..people.older.than.60,Median.income,X..African.Americans,X..Hispanic.Americans,X..Male,Population.density,Employment.density,X..hot.spots.1000.people,Hospital.beds.1000.people,ICUs.1000.people,X..contact.tracing.workers.1000.people,New.cases.1000.people,Active.cases.1000.people,X.days..decreasing.COVID.cases,X..hospital.bed.utilization,Testing.capacity,Tests.done.1000.people,X..ICU.utilization,Ventilator.shortage,Imported.COVID.cases,COVID.exposure.1000.people,X.days..decreasing.ILI.cases,Unemployment.claims.1000.people,Unemployment.rate,X..working.from.home,Cumulative.inflation.rate,COVID.death.rate
0,2020-03-03,Montana,30001,0,Beaverhead County,9453,0.0,1.139,86,9.0,7.0,0.0,0.0,-2.0,0.0,Montana,0.654314,30001,Beaverhead County,MT,9453,30001,Beaverhead County,30,17,20,3.33,20.4,6.6,34.1,0.68,2.65,0,9404,13.2,2020-03-03,0.02,29,44809,0.1,4.5,50.55,2,1,177,3.26,0.32,0.113,0.0,0.00,39,46.600000,0.0,0.00,0.00,0,0,0.00,0,0.7,3.5,6.6,0.3,0.00
1,2020-03-04,Montana,30001,0,Beaverhead County,9453,0.0,1.480,113,14.0,7.0,0.0,0.0,3.0,0.0,Montana,0.654314,30001,Beaverhead County,MT,9453,30001,Beaverhead County,30,13,16,3.44,21.3,8.4,36.1,0.71,2.73,0,9404,16.7,2020-03-04,0.02,29,44809,0.1,4.5,50.55,2,1,177,3.26,0.32,0.113,0.0,0.00,40,46.600000,0.0,0.00,0.00,0,0,0.00,0,0.7,3.5,6.6,0.3,0.00
2,2020-04-21,Montana,30001,0,Beaverhead County,9453,0.0,1.107,84,0.0,0.0,0.0,0.0,-22.0,0.0,Montana,0.654314,30001,Beaverhead County,MT,9453,30001,Beaverhead County,30,22,13,2.90,16.3,6.9,25.2,0.52,2.39,0,9404,2.1,2020-04-21,0.02,29,44809,0.1,4.5,50.55,2,1,177,3.26,0.32,0.113,0.0,0.38,11,47.156862,3.9,10.58,1.49,4,2,0.08,49,6.2,11.2,24.1,0.0,14.65
3,2020-04-22,Montana,30001,0,Beaverhead County,9453,0.0,1.286,98,0.0,0.0,0.0,0.0,-24.0,0.0,Montana,0.654314,30001,Beaverhead County,MT,9453,30001,Beaverhead County,30,13,16,3.18,21.3,7.7,40.9,0.62,2.56,0,9404,9.4,2020-04-22,0.02,29,44809,0.1,4.5,50.55,2,1,177,3.26,0.32,0.113,0.0,0.38,12,47.025863,3.8,10.90,1.19,3,2,0.08,49,6.2,11.2,24.1,0.0,17.01
4,2020-04-23,Montana,30001,0,Beaverhead County,9453,0.0,1.730,132,0.0,0.0,0.0,0.0,-30.0,0.0,Montana,0.654314,30001,Beaverhead County,MT,9453,30001,Beaverhead County,30,17,18,3.33,19.8,10.7,32.4,0.54,2.79,0,9404,19.2,2020-04-23,0.02,29,44809,0.1,4.5,50.55,2,1,177,3.26,0.32,0.113,0.0,0.37,13,46.894867,3.7,11.18,0.89,2,3,0.08,49,6.2,11.2,24.1,0.0,16.97


In [9]:
print(All_state_superset_data.date.min(),All_state_superset_data.date.max())

2020-03-01 2020-12-28


In [10]:
All_state_superset_data.head(1)

,Date,Province_State,FIPS,New cases,County Name_x,population_x,mask_rule_severity,m50,m50_index,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,State_Name,PopDensity,countyFIPS,County Name_y,State,population_y,CTFIPS,CTNAME,STFIPS,Social.distancing.index,X..staying.home,Trips.person,X..out.of.county.trips,X..out.of.state.trips,Miles.person,Work.trips.person,Non.work.trips.person,New.COVID.cases,Population,X..change.in.consumption,date,Transit.mode.share,X..people.older.than.60,Median.income,X..African.Americans,X..Hispanic.Americans,X..Male,Population.density,Employment.density,X..hot.spots.1000.people,Hospital.beds.1000.people,ICUs.1000.people,X..contact.tracing.workers.1000.people,New.cases.1000.people,Active.cases.1000.people,X.days..decreasing.COVID.cases,X..hospital.bed.utilization,Testing.capacity,Tests.done.1000.people,X..ICU.utilization,Ventilator.shortage,Imported.COVID.cases,COVID.exposure.1000.people,X.days..decreasing.ILI.cases,Unemployment.claims.1000.people,Unemployment.rate,X..working.from.home,Cumulative.inflation.rate,COVID.death.rate
0,2020-03-03,Montana,30001,0,Beaverhead County,9453,0.0,1.139,86,9.0,7.0,0.0,0.0,-2.0,0.0,Montana,0.654314,30001,Beaverhead County,MT,9453,30001,Beaverhead County,30,17,20,3.33,20.4,6.6,34.1,0.68,2.65,0,9404,13.2,2020-03-03,0.02,29,44809,0.1,4.5,50.55,2,1,177,3.26,0.32,0.113,0.0,0.0,39,46.6,0.0,0.0,0.0,0,0,0.0,0,0.7,3.5,6.6,0.3,0.0


In [11]:
All_state_superset_data.columns

Index(['Date', 'Province_State', 'FIPS', 'New cases', 'County Name_x',
       'population_x', 'mask_rule_severity', 'm50', 'm50_index',
       'retail_and_recreation_percent_change_from_baseline',
       'grocery_and_pharmacy_percent_change_from_baseline',
       'parks_percent_change_from_baseline',
       'transit_stations_percent_change_from_baseline',
       'workplaces_percent_change_from_baseline',
       'residential_percent_change_from_baseline', 'State_Name', 'PopDensity',
       'countyFIPS', 'County Name_y', 'State', 'population_y', 'CTFIPS',
       'CTNAME', 'STFIPS', 'Social.distancing.index', 'X..staying.home',
       'Trips.person', 'X..out.of.county.trips', 'X..out.of.state.trips',
       'Miles.person', 'Work.trips.person', 'Non.work.trips.person',
       'New.COVID.cases', 'Population', 'X..change.in.consumption', 'date',
       'Transit.mode.share', 'X..people.older.than.60', 'Median.income',
       'X..African.Americans', 'X..Hispanic.Americans', 'X..Male',
       '

In [12]:
All_state_superset_data = All_state_superset_data.rename({'County Name_x': 'County Name','PopDensity_x':'PopDensity'}, axis=1)  # new method

In [13]:
All_state_superset_data.columns

Index(['Date', 'Province_State', 'FIPS', 'New cases', 'County Name',
       'population_x', 'mask_rule_severity', 'm50', 'm50_index',
       'retail_and_recreation_percent_change_from_baseline',
       'grocery_and_pharmacy_percent_change_from_baseline',
       'parks_percent_change_from_baseline',
       'transit_stations_percent_change_from_baseline',
       'workplaces_percent_change_from_baseline',
       'residential_percent_change_from_baseline', 'State_Name', 'PopDensity',
       'countyFIPS', 'County Name_y', 'State', 'population_y', 'CTFIPS',
       'CTNAME', 'STFIPS', 'Social.distancing.index', 'X..staying.home',
       'Trips.person', 'X..out.of.county.trips', 'X..out.of.state.trips',
       'Miles.person', 'Work.trips.person', 'Non.work.trips.person',
       'New.COVID.cases', 'Population', 'X..change.in.consumption', 'date',
       'Transit.mode.share', 'X..people.older.than.60', 'Median.income',
       'X..African.Americans', 'X..Hispanic.Americans', 'X..Male',
       'Po

## Compute `New Cases/100k population`

In [14]:

All_state_superset_data['New Cases/100k population'] = All_state_superset_data['New cases']*100000/ All_state_superset_data['Population']
All_state_superset_data.tail(2)

,Date,Province_State,FIPS,New cases,County Name,population_x,mask_rule_severity,m50,m50_index,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,State_Name,PopDensity,countyFIPS,County Name_y,State,population_y,CTFIPS,CTNAME,STFIPS,Social.distancing.index,X..staying.home,Trips.person,X..out.of.county.trips,X..out.of.state.trips,Miles.person,Work.trips.person,Non.work.trips.person,New.COVID.cases,Population,X..change.in.consumption,date,Transit.mode.share,X..people.older.than.60,Median.income,X..African.Americans,X..Hispanic.Americans,X..Male,Population.density,Employment.density,X..hot.spots.1000.people,Hospital.beds.1000.people,ICUs.1000.people,X..contact.tracing.workers.1000.people,New.cases.1000.people,Active.cases.1000.people,X.days..decreasing.COVID.cases,X..hospital.bed.utilization,Testing.capacity,Tests.done.1000.people,X..ICU.utilization,Ventilator.shortage,Imported.COVID.cases,COVID.exposure.1000.people,X.days..decreasing.ILI.cases,Unemployment.claims.1000.people,Unemployment.rate,X..working.from.home,Cumulative.inflation.rate,COVID.death.rate,New Cases/100k population
3101379,2020-12-27,Kansas,20209,0,Wyandotte County,165429,0.0,3.460,27,-20.0,-16.0,16.0,-48.0,-7.0,5.0,Kansas,418.452206,20209,Wyandotte County,KS,165429,20209,Wyandotte County,20,48,34,2.70,48.5,22.3,41.0,0.20,2.50,0,165324,-15.9,2020-12-27,0.99,17,44873,22.1,28.4,49.47,1056,504,103,4.25,0.45,0.003,0.0000,19.52,10,50.505875,21.8,330.48,19.54,130,3077,57.21,0,7.9,14.6,25.0,1.8,1.5,0.000000
3101380,2020-12-28,Kansas,20209,349,Wyandotte County,165429,0.0,7.497,59,-5.0,-3.0,-9.0,-49.0,-33.0,10.0,Kansas,418.452206,20209,Wyandotte County,KS,165429,20209,Wyandotte County,20,28,26,3.35,50.7,24.2,41.7,0.44,2.91,349,165324,-2.2,2020-12-28,0.99,17,44873,22.1,28.4,49.47,1056,504,103,4.25,0.45,0.003,0.7037,21.55,0,50.709038,21.9,338.59,8.97,132,4370,57.21,0,7.9,14.6,25.0,1.8,1.5,211.100627


# Compute `Rolling Average /100k' for all counties

In [15]:
col_idx  = All_state_superset_data.columns.get_loc("New Cases/100k population")
print(col_idx)

64


In [16]:
df = pd.DataFrame(columns=['Date', 'Province_State', 'FIPS', 'New cases', 'County Name',
       'mask_rule_severity', 'm50', 'm50_index',
       'retail_and_recreation_percent_change_from_baseline',
       'grocery_and_pharmacy_percent_change_from_baseline',
       'parks_percent_change_from_baseline',
       'transit_stations_percent_change_from_baseline',
       'workplaces_percent_change_from_baseline',
       'residential_percent_change_from_baseline', 'PopDensity', 'State_Name',
       'State', 'CTFIPS', 'CTNAME', 'STFIPS', 'Social.distancing.index',
       'X..staying.home', 'Trips.person', 'X..out.of.county.trips',
       'X..out.of.state.trips', 'Miles.person', 'Work.trips.person',
       'Non.work.trips.person', 'New.COVID.cases', 'Population',
       'X..change.in.consumption', 'Transit.mode.share',
       'X..people.older.than.60', 'Median.income', 'X..African.Americans',
       'X..Hispanic.Americans', 'X..Male', 'Employment.density',
       'X..hot.spots.1000.people', 'Hospital.beds.1000.people',
       'ICUs.1000.people', 'X..contact.tracing.workers.1000.people',
       'New.cases.1000.people', 'Active.cases.1000.people',
       'X.days..decreasing.COVID.cases', 'X..hospital.bed.utilization',
       'Testing.capacity', 'Tests.done.1000.people', 'X..ICU.utilization',
       'Ventilator.shortage', 'Imported.COVID.cases',
       'COVID.exposure.1000.people', 'X.days..decreasing.ILI.cases',
       'Unemployment.claims.1000.people', 'Unemployment.rate',
       'X..working.from.home', 'Cumulative.inflation.rate',
       'COVID.death.rate','New Cases/100k population','rolling_avg_new_cases/100k'])

In [17]:
All_states_counties_num = All_state_superset_data['County Name'].nunique()
All_states_counties_num

1541

In [18]:
from tqdm import tqdm

In [19]:
All_states_counties = All_state_superset_data['County Name'].unique()

for i in tqdm(range(All_states_counties_num)):
  county = All_states_counties[i]
  county_data = All_state_superset_data[All_state_superset_data['County Name'] == county]
  #print(county_data.iloc[:,15])
  county_data['rolling_avg_new_cases/100k'] = county_data.iloc[:,col_idx].rolling(window=7).mean()
  df = df.append(county_data)

df.head()

  0%|          | 0/1541 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
100%|██████████| 1541/1541 [1:48:59<00:00,  4.24s/it]


,Date,Province_State,FIPS,New cases,County Name,mask_rule_severity,m50,m50_index,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,PopDensity,State_Name,State,CTFIPS,CTNAME,STFIPS,Social.distancing.index,X..staying.home,Trips.person,X..out.of.county.trips,X..out.of.state.trips,Miles.person,Work.trips.person,Non.work.trips.person,New.COVID.cases,Population,X..change.in.consumption,Transit.mode.share,X..people.older.than.60,Median.income,X..African.Americans,X..Hispanic.Americans,X..Male,Employment.density,X..hot.spots.1000.people,Hospital.beds.1000.people,ICUs.1000.people,X..contact.tracing.workers.1000.people,New.cases.1000.people,Active.cases.1000.people,X.days..decreasing.COVID.cases,X..hospital.bed.utilization,Testing.capacity,Tests.done.1000.people,X..ICU.utilization,Ventilator.shortage,Imported.COVID.cases,COVID.exposure.1000.people,X.days..decreasing.ILI.cases,Unemployment.claims.1000.people,Unemployment.rate,X..working.from.home,Cumulative.inflation.rate,COVID.death.rate,New Cases/100k population,rolling_avg_new_cases/100k,population_x,countyFIPS,County Name_y,population_y,date,Population.density
0,2020-03-03,Montana,30001,0,Beaverhead County,0.0,1.139,86,9.0,7.0,0.0,0.0,-2.0,0.0,0.654314,Montana,MT,30001,Beaverhead County,30,17,20,3.33,20.4,6.6,34.1,0.68,2.65,0,9404,13.2,0.02,29,44809,0.1,4.5,50.55,1,177,3.26,0.32,0.113,0.0,0.00,39,46.600000,0.0,0.00,0.00,0,0,0.00,0,0.7,3.5,6.6,0.3,0.00,0.0,NaN,9453.0,30001.0,Beaverhead County,9453.0,2020-03-03,2.0
1,2020-03-04,Montana,30001,0,Beaverhead County,0.0,1.480,113,14.0,7.0,0.0,0.0,3.0,0.0,0.654314,Montana,MT,30001,Beaverhead County,30,13,16,3.44,21.3,8.4,36.1,0.71,2.73,0,9404,16.7,0.02,29,44809,0.1,4.5,50.55,1,177,3.26,0.32,0.113,0.0,0.00,40,46.600000,0.0,0.00,0.00,0,0,0.00,0,0.7,3.5,6.6,0.3,0.00,0.0,NaN,9453.0,30001.0,Beaverhead County,9453.0,2020-03-04,2.0
2,2020-04-21,Montana,30001,0,Beaverhead County,0.0,1.107,84,0.0,0.0,0.0,0.0,-22.0,0.0,0.654314,Montana,MT,30001,Beaverhead County,30,22,13,2.90,16.3,6.9,25.2,0.52,2.39,0,9404,2.1,0.02,29,44809,0.1,4.5,50.55,1,177,3.26,0.32,0.113,0.0,0.38,11,47.156862,3.9,10.58,1.49,4,2,0.08,49,6.2,11.2,24.1,0.0,14.65,0.0,NaN,9453.0,30001.0,Beaverhead County,9453.0,2020-04-21,2.0
3,2020-04-22,Montana,30001,0,Beaverhead County,0.0,1.286,98,0.0,0.0,0.0,0.0,-24.0,0.0,0.654314,Montana,MT,30001,Beaverhead County,30,13,16,3.18,21.3,7.7,40.9,0.62,2.56,0,9404,9.4,0.02,29,44809,0.1,4.5,50.55,1,177,3.26,0.32,0.113,0.0,0.38,12,47.025863,3.8,10.90,1.19,3,2,0.08,49,6.2,11.2,24.1,0.0,17.01,0.0,NaN,9453.0,30001.0,Beaverhead County,9453.0,2020-04-22,2.0
4,2020-04-23,Montana,30001,0,Beaverhead County,0.0,1.730,132,0.0,0.0,0.0,0.0,-30.0,0.0,0.654314,Montana,MT,30001,Beaverhead County,30,17,18,3.33,19.8,10.7,32.4,0.54,2.79,0,9404,19.2,0.02,29,44809,0.1,4.5,50.55,1,177,3.26,0.32,0.113,0.0,0.37,13,46.894867,3.7,11.18,0.89,2,3,0.08,49,6.2,11.2,24.1,0.0,16.97,0.0,NaN,9453.0,30001.0,Beaverhead County,9453.0,2020-04-23,2.0


## Filtered columns

In [20]:
required_cols = ['Date', 'Province_State', 'FIPS', 'New cases', 'County Name',
       'mask_rule_severity', 'm50', 'm50_index',
       'retail_and_recreation_percent_change_from_baseline',
       'grocery_and_pharmacy_percent_change_from_baseline',
       'parks_percent_change_from_baseline',
       'transit_stations_percent_change_from_baseline',
       'workplaces_percent_change_from_baseline',
       'residential_percent_change_from_baseline', 'PopDensity', 
       'State_Name', 'State',
       'CTFIPS', 'CTNAME', 'STFIPS', 'Social.distancing.index',
       'X..staying.home', 'Trips.person', 'X..out.of.county.trips',
       'X..out.of.state.trips', 'Miles.person', 'Work.trips.person',
       'Non.work.trips.person', 'New.COVID.cases', 'Population',
       'X..change.in.consumption', 'Transit.mode.share',
       'X..people.older.than.60', 'Median.income', 'X..African.Americans',
       'X..Hispanic.Americans', 'X..Male',
       'Employment.density', 'X..hot.spots.1000.people',
       'Hospital.beds.1000.people', 'ICUs.1000.people',
       'X..contact.tracing.workers.1000.people', 'New.cases.1000.people',
       'Active.cases.1000.people', 'X.days..decreasing.COVID.cases',
       'X..hospital.bed.utilization', 'Testing.capacity',
       'Tests.done.1000.people', 'X..ICU.utilization', 'Ventilator.shortage',
       'Imported.COVID.cases', 'COVID.exposure.1000.people',
       'X.days..decreasing.ILI.cases', 'Unemployment.claims.1000.people',
       'Unemployment.rate', 'X..working.from.home',
       'Cumulative.inflation.rate', 'COVID.death.rate','New Cases/100k population','rolling_avg_new_cases/100k']

In [25]:
len(required_cols)

60

In [21]:
All_state_superset_data_filtered = df[required_cols]


In [22]:
All_state_superset_data_filtered.columns

Index(['Date', 'Province_State', 'FIPS', 'New cases', 'County Name',
       'mask_rule_severity', 'm50', 'm50_index',
       'retail_and_recreation_percent_change_from_baseline',
       'grocery_and_pharmacy_percent_change_from_baseline',
       'parks_percent_change_from_baseline',
       'transit_stations_percent_change_from_baseline',
       'workplaces_percent_change_from_baseline',
       'residential_percent_change_from_baseline', 'PopDensity', 'State_Name',
       'State', 'CTFIPS', 'CTNAME', 'STFIPS', 'Social.distancing.index',
       'X..staying.home', 'Trips.person', 'X..out.of.county.trips',
       'X..out.of.state.trips', 'Miles.person', 'Work.trips.person',
       'Non.work.trips.person', 'New.COVID.cases', 'Population',
       'X..change.in.consumption', 'Transit.mode.share',
       'X..people.older.than.60', 'Median.income', 'X..African.Americans',
       'X..Hispanic.Americans', 'X..Male', 'Employment.density',
       'X..hot.spots.1000.people', 'Hospital.beds.1000.peop

# All States Save to csv file 

In [26]:
from datetime import datetime
location = '/content/drive/Shared drives/CMPE 295- Master Project/projectdata-2021/'
filename = 'AllStates_Superset_dataset_'+datetime.now().strftime("%b%d")+'.csv' #%Y%m%d
print(filename)
All_state_superset_data_filtered.to_csv(location+filename,index=False)

AllStates_Superset_dataset_Feb21.csv


# CA Save to csv file 

In [ ]:
ca_data = CA_superset_data_filtered[CA_superset_data_filtered['State'] == 'CA']
print(ca_data.shape)
ca_data.head(2)

(21986, 58)


,Date,Province_State,FIPS,New cases,County Name,mask_rule_severity,m50,m50_index,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,PopDensity,State_Name,State,CTFIPS,CTNAME,STFIPS,Social.distancing.index,X..staying.home,Trips.person,X..out.of.county.trips,X..out.of.state.trips,Miles.person,Work.trips.person,Non.work.trips.person,New.COVID.cases,Population,X..change.in.consumption,Transit.mode.share,X..people.older.than.60,Median.income,X..African.Americans,X..Hispanic.Americans,X..Male,Employment.density,X..hot.spots.1000.people,Hospital.beds.1000.people,ICUs.1000.people,X..contact.tracing.workers.1000.people,New.cases.1000.people,Active.cases.1000.people,X.days..decreasing.COVID.cases,X..hospital.bed.utilization,Testing.capacity,Tests.done.1000.people,X..ICU.utilization,Ventilator.shortage,Imported.COVID.cases,COVID.exposure.1000.people,X.days..decreasing.ILI.cases,Unemployment.claims.1000.people,Unemployment.rate,X..working.from.home,Cumulative.inflation.rate,COVID.death.rate
369128,2020-03-01,California,6001,0,Alameda County,0.0,3.684,58,6.0,9.0,15.0,1.0,4.0,0.0,860.757075,California,CA,6001,Alameda County,6,37,26,2.93,25.4,0.4,36.8,0.25,2.68,0,1666753,-5.1,15.04,19,92574,10.5,22.5,49.11,1155,131,2.05,0.22,NaN,0.0,0.0,37,54.0,0.0,0.0,0.0,0,0,0.05,35,1.1,5.3,5.7,0.6,0.0
369129,2020-03-02,California,6001,0,Alameda County,0.0,6.182,98,4.0,7.0,7.0,-1.0,3.0,0.0,860.757075,California,CA,6001,Alameda County,6,17,20,3.38,28.2,0.4,38.1,0.63,2.74,0,1666753,-2.9,15.04,19,92574,10.5,22.5,49.11,1155,131,2.05,0.22,NaN,0.0,0.0,38,54.0,0.0,0.0,0.0,0,0,0.05,35,1.1,5.3,5.7,0.6,0.0


In [ ]:
from datetime import datetime
location = '/content/drive/Shared drives/CMPE 295- Master Project/projectdata-2021/'
filename = 'CA_Superset_dataset_'+datetime.now().strftime("%b%d")+'.csv' #%Y%m%d
print(filename)
ca_data.to_csv(location+filename,index=False)

CA_Superset_dataset_Feb12.csv
